In [2]:
# importando bibliotecas para todos os algoritmos

import warnings
import numpy as np 
import pylab as pyl 
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

%pylab inline
%matplotlib inline 

Populating the interactive namespace from numpy and matplotlib


In [3]:
warnings.filterwarnings('ignore')

In [49]:
# importando o dataset
df = pd.read_csv('../dataset/data-set-clube-new-v2.csv', sep=';')

In [50]:
df.head(1)

,id_socio,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
0,38810,CANCELADO,24,0,0,0


In [51]:
df.tail(1)

,id_socio,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
3026,49083,ATIVO,51,4,12,15


In [52]:
# Removendo a coluna id_socio (não usaremos ela por enquanto)
df.drop('id_socio', inplace=True, axis=1)

In [54]:
# Verificando se existem valores nulos
df.isnull().values.any()

False

Após o tratamento abaixo, o STATUS ficará: 

- ATIVO = 0 
- CANCELADO = 1

In [55]:
# Passando a coluna STATUS para o tipo inteiro
label_encoder = preprocessing.LabelEncoder()
df['status'] = label_encoder.fit_transform(df['status'])

In [56]:
# Definindo as classes
status_map = {True : 1, False : 0}

In [57]:
# Aplicando o mapeamento ao dataset
df['status'] = df['status'].map(status_map)

In [58]:
df.head(1)

,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
0,1,24,0,0,0


In [60]:
df.tail(1)

,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
3026,0,51,4,12,15


In [64]:
# Verificando como os dados estão distribuídos
num_true = len(df.loc[df['status'] == True])
num_false = len(df.loc[df['status'] == False])
print("Número de Status Cancelados: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Status Ativos    : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

Número de Status Cancelados: 771 (25.47%)
Número de Status Ativos    : 2256 (74.53%)


## Spliting

70% para dados de treino e 30% para dados de teste

In [66]:
# Seleção de variáveis preditoras (Feature Selection)
atributos = ['qtde_em_aberto', 'qtde_em_dia', 'qtde_em_atraso', 'qtde_frequencia_ano']

In [67]:
# Variável a ser prevista
atrib_prev = ['status']

In [68]:
# Criando objetos
X = df[atributos].values
Y = df[atrib_prev].values

In [73]:
# Definindo a taxa de split
split_test_size = 0.30

In [74]:
# Criando dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)

In [75]:
# Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(df.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(df.index)) * 100))

69.97% nos dados de treino
30.03% nos dados de teste


### Verificando o Split

In [78]:
print("Original True : {0} ({1:0.2f}%)".format(len(df.loc[df['status'] == 1]), 
                                               (len(df.loc[df['status'] ==1])/len(df.index) * 100)))

print("Original False : {0} ({1:0.2f}%)".format(len(df.loc[df['status'] == 0]), 
                                               (len(df.loc[df['status'] == 0])/len(df.index) * 100)))
print("")
print("Training True : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 1]), 
                                               (len(Y_treino[Y_treino[:] == 1])/len(Y_treino) * 100)))

print("Training False : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 0]), 
                                               (len(Y_treino[Y_treino[:] == 0])/len(Y_treino) * 100)))
print("")
print("Test True : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 1]), 
                                               (len(Y_teste[Y_teste[:] == 1])/len(Y_teste) * 100)))

print("Test False : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 0]), 
                                               (len(Y_teste[Y_teste[:] == 0])/len(Y_teste) * 100)))

Original True : 771 (25.47%)
Original False : 2256 (74.53%)

Training True : 530 (25.02%)
Training False : 1588 (74.98%)

Test True : 241 (26.51%)
Test False : 668 (73.49%)
